In [1]:
import os
import httpx
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import requests
import numpy as np
import import_ipynb
import ollama
from evaluate import load
from rouge import Rouge


In [2]:
import import_ipynb
import importlib
import Eval_Metrics
eval_metrics = Eval_Metrics.Evaluation_Metrics()
# importlib.reload(Eval_Metrics)
# eval_metrics = Eval_Metrics.Evaluation_Metrics()

#### Chosen LLM

In [3]:
model = "llama3.1:latest"

### Getting Ranking List Prompt Dataset to be used in Ranking List LLM Evaluation

In [4]:
prompt_examples_df = pd.read_excel(r"C:\Users\flame\CP5105-AI-Powered-Product-and-Service-Review-Summarizer-and-Ranking-System\BackEnd\All Prompt Examples\Ranking List Prompt Dataset.xlsx")


In [5]:
prompt_examples_df['Modified_Multiple_Generated_Summaries'] = prompt_examples_df['Modified_Multiple_Generated_Summaries'].map(lambda a: a.replace('\n',' '))

In [6]:
prompt_examples_df.columns

Index(['Query', 'Modified_1_Generated_Summary',
       'Modified_Multiple_Generated_Summaries', 'Model_Ranking_List_Output',
       'Model_Ranking_List_for_Manual_Checking', 'Prompt', 'Unnamed: 6'],
      dtype='object')

#### Getting Data to be used as Prompt Examples from dataset

In [7]:
promptengin_df = prompt_examples_df[prompt_examples_df['Prompt']=='Use for Prompt'].reset_index(drop=True)
promptengin_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt,Unnamed: 6
0,What are the worst mobile phone games released...,The gaming industry has been flooded with new ...,Article #1: The gaming industry has been flood...,Here is the ranking list for the worst mobile ...,1: Robot Rebellion\n2. Galactic Conquest\nRest...,Use for Prompt,NaN
1,What are the top 5 best sci-fi movies of all t...,This is a question that has been debated by fi...,Article #1: This is a question that has been ...,Here is the ranking list for the top 5 best sc...,1: Blade Runner\n2: The Matrix\n3: A Space Ody...,Use for Prompt,NaN
2,What are the 3 worst games consoles of all time?,The Atari Jaguar is often cited as one of the ...,Article #1: The Atari Jaguar is often cited a...,Here is the ranking list for the 3 worst games...,"1,2: Atari Jaguar, Nintendo Virtual Boy\n3: Se...",Use for Prompt,NaN
3,What are the 5 worst board games?,Determining the worst board games can be subje...,Article #1: Determining the worst board games ...,Here is the ranking list for the 5 worst board...,1: Monikers\n2: Don’t Wake Daddy\n3: Battleshi...,Use for Prompt,NaN
4,What are the best printer models in 2024?,Choosing the right printer can be overwhelming...,Article #1: Choosing the right printer can be ...,Here is the ranking list for the best printer ...,1. Epson Expression Photo HD XP-15000\n2. Xero...,Use for Prompt,NaN


#### Getting Data to be used for Ranking List LLM testing from dataset

In [8]:
testprompt_df = prompt_examples_df[prompt_examples_df['Prompt'].isna()==True].reset_index(drop=True)
testprompt_df

,Query,Modified_1_Generated_Summary,Modified_Multiple_Generated_Summaries,Model_Ranking_List_Output,Model_Ranking_List_for_Manual_Checking,Prompt,Unnamed: 6
0,What are the top car models in 2024?,The automotive industry is constantly evolving...,Article #1: The automotive industry is constan...,Here is the ranking list for the top car model...,"1,2: Toyota Camry , Honda Civic Type R\n3,4: H...",NaN,What are the top car models in 2024?
1,What are the 3 worst laptop brands in 2024?,The ranking of laptop brands can vary dependin...,Article #1: The ranking of laptop brands can v...,Here is the ranking list for the 3 worst lapto...,1: Acer\n2: Lenovo\n3: Asus,NaN,What are the 3 worst laptop brands in 2024?
2,What are the cheapest but decent laptop models...,"When it comes to budget-friendly laptops, ther...",Article #1: When it comes to budget-friendly l...,Here is the ranking list for the top budget la...,"1: Lenovo IdeaPad 330S\n2,3, 4 ,5: Acer Aspire...",NaN,What are the cheapest but decent laptop models...
3,What are the top sunscreen brands?,Sunscreen is a vital part of our daily skincar...,Article #1: Sunscreen is a vital part of our ...,Here is the ranking list for the top sunscreen...,"1. Neutrogena\n2. Hawaiian Tropic\n3,4,5: La R...",NaN,What are the top sunscreen brands?
4,What are the most interesting and top rated bo...,The year 2024 is expected to bring a diverse r...,Article #1: The year 2024 is expected to bring...,Here is the ranking list for the most interest...,"1,2: The It Girl, The Memory Keeper\'s Daughte...",NaN,What are the most interesting and top rated bo...
5,What are the most hated and worst books releas...,"Based on user ratings on Goodreads, some of th...",Article #1: Based on user ratings on Goodreads...,Here is the ranking list for the most hated an...,"1,2: No Going Back , Crown of Starlight\n3,4 :...",NaN,What are the most hated and worst books releas...
6,What are the 5 worst horror movies of all time?,The assessment of the worst horror movies is s...,Article #1: The assessment of the worst horror...,Here is the ranking list for the 5 worst horro...,1: Devil\n2: Birdemic: Shock and Terror (2010)...,NaN,What are the 5 worst horror movies of all time?
7,What are the 3 best games consoles of all time?,"Determining the ""best"" gaming consoles is subj...","Article #1: Determining the ""best"" gaming cons...",Here is the ranking list for the top 3 best ga...,"1,2: PlayStation 2 (PS2) , Nintendo Entertainm...",NaN,What are the 3 best games consoles of all time?
8,What are the 5 best board games for children?,Choosing the right board game can be a challen...,Article #1: Choosing the right board game can ...,Here is the ranking list for the 5 best board ...,"1,2,3,4 : Candy Land, Chutes and Ladders, Mono...",NaN,What are the 5 best board games for children?
9,What are the worst mobile phones in 2024?,The ranking of the worst mobile phones can be ...,Article #1: The ranking of the worst mobile ph...,Here is the ranking list for the worst mobile ...,"1. Motorola Defy Pro X\n2,3: HTC One M7, Samsu...",NaN,What are the worst mobile phones in 2024?


#### Ranking List response generation function for Zero Step COT SCT text input

In [14]:
def ZERO_STEP_SYSTEM_COT_SINGLE_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt + f'''
            Let\'s think step by step.
            1) Identify the item names relevant to the {query} in the user prompt text.
            2) Calculate number of times each item name is mentioned in the user prompt text.
            3) Create an ordered ranking list ordered from item name that appeared most amount of times in the user prompt text to the item name that appeared least amount of times in the user prompt text.
            4) Provide a description for each item name in the ordered ranking list.
            '''
        },
        {
            'role': 'user',
            'content': f'''In the context of "{query}" , generate an ordered and ranked list from the following text : '''
            + text,
        },

        ]
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Step SCT text input

In [13]:
def SINGLE_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },

        ]
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.96 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [15]:
def SINGLE_0_96T_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "temperature": 0.96
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.64 Temp , 0.9 Top P , 40 Top K ) SYS with SCT text input

In [16]:
def SINGLE_0_64T_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "temperature": 0.64
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.9 Top P , 48 Top K ) SYS with SCT text input

In [17]:
def SINGLE_48topk_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_k": 48
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.9 Top P , 32 Top K ) SYS with SCT text input

In [18]:
def SINGLE_32topk_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_k": 32
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 1.08 Top P , 40 Top K ) SYS with SCT text input

In [19]:
def SINGLE_1_08topp_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": 1.08
        }
        )
    
    
    report = response['message']['content']
    
    return report

#### Ranking List response generation function for Zero Shot (0.8 Temp , 0.72 Top P , 48 Top K ) SYS with SCT text input

In [20]:
def SINGLE_0_72topp_generate_report_with_query(text, model,query,prompt):
               

    
    response = ollama.chat(
        model=model, 
        messages=[
        {
            'role': 'system',
            'content': prompt
        },
        {
            'role': 'user',
            'content': f'In the context of "{query}" , generate an ordered and ranked list from the following text : ' 
            + text,
        },
        ],
        options = {
            "top_p": 0.72
        }
        )
    
    
    report = response['message']['content']
    
    return report

##### Dataframe Initialization

In [21]:
zero_shot_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','avg_score'])

##### Chosen Model

In [22]:
model = "llama3.1:latest"

### Evaluation

### Zero Shot Prompt

In [23]:
query =''

In [24]:
def zero_shot_prompt_SINGLE(query):
        zero_shot_prompt =  f'''You are a reviewer generating an ordered and numbered ranked list based on {query} only. \
                Each item in the ordered list should be ordered from item that appeared most amount of times in the user prompt text to the item that appeared least amount of times in the user prompt text. \
                Each item in the ordered list should have the item name and the item details from the user prompt text. \
                Bold each item on the ranking list in the response.\
                To make text bold, enclose it with double asterisks (**). Example: **text** becomes bold.\
                '''
        return zero_shot_prompt

#### Sensitivity Analysis

#### For Default

In [41]:
zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               
               avg_score,
               ]
    
    zero_shot_single_response_df.loc[-1] = ref_row
    zero_shot_single_response_df.index = zero_shot_single_response_df.index + 1  #shift index
    zero_shot_single_response_df = zero_shot_single_response_df.sort_index()  #sort by index

In [42]:
zero_shot_single_response_df.to_csv('FINETUNE_zero_shot_single_response_df_2.csv')

#### For Temp = 0.96

In [39]:
SINGLE_0_96T_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_0_96T_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_0_96T_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_0_96T_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
                response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_0_96T_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_0_96T_zero_shot_single_response_df.index = SINGLE_0_96T_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_0_96T_zero_shot_single_response_df = SINGLE_0_96T_zero_shot_single_response_df.sort_index()  #sort by index

In [40]:
SINGLE_0_96T_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_0_96T_zero_shot_single_response_df_2.csv')

#### For Temp = 0.64

In [29]:
SINGLE_0_64T_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_0_64T_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_0_64T_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_0_64T_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_0_64T_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_0_64T_zero_shot_single_response_df.index = SINGLE_0_64T_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_0_64T_zero_shot_single_response_df = SINGLE_0_64T_zero_shot_single_response_df.sort_index()  #sort by index

In [30]:
SINGLE_0_64T_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_0_64T_zero_shot_single_response_df_2.csv')

#### For Top K = 48

In [31]:
SINGLE_48topk_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_48topk_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_48topk_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_48topk_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_48topk_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_48topk_zero_shot_single_response_df.index = SINGLE_48topk_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_48topk_zero_shot_single_response_df = SINGLE_48topk_zero_shot_single_response_df.sort_index()  #sort by index

In [32]:
SINGLE_48topk_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_48topk_zero_shot_single_response_df_2.csv')

#### For Top K = 32

In [33]:
SINGLE_32topk_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_32topk_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_32topk_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_32topk_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
                response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_32topk_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_32topk_zero_shot_single_response_df.index = SINGLE_32topk_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_32topk_zero_shot_single_response_df = SINGLE_32topk_zero_shot_single_response_df.sort_index()  #sort by index

In [34]:
SINGLE_32topk_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_32topk_zero_shot_single_response_df_2.csv')

#### For Top P = 1.08

In [35]:
SINGLE_1_08topp_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_1_08topp_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_1_08topp_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_1_08topp_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
                response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_1_08topp_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_1_08topp_zero_shot_single_response_df.index = SINGLE_1_08topp_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_1_08topp_zero_shot_single_response_df = SINGLE_1_08topp_zero_shot_single_response_df.sort_index()  #sort by index

In [36]:
SINGLE_1_08topp_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_1_08topp_zero_shot_single_response_df_2.csv')

#### For Top P = 0.72

In [37]:
SINGLE_0_72topp_zero_shot_single_response_df = pd.DataFrame(columns=['query','model_response','zero_shot_response_1','zero_shot_response_1_score','zero_shot_response_2','zero_shot_response_2_score','zero_shot_response_3','zero_shot_response_3_score','avg_score'])

for index ,row in testprompt_df.iterrows():
    text = row.Modified_1_Generated_Summary
    query = row.Query
    prompt = zero_shot_prompt_SINGLE(query)
    model_response = row.Model_Ranking_List_Output
    zero_shot_response_1_score = ''
    zero_shot_response_2_score = ''
    zero_shot_response_3_score = ''
    avg_score = ''
    
    response_text_1 = SINGLE_0_72topp_generate_report_with_query(text, model,query,prompt)
    response_text_2 = SINGLE_0_72topp_generate_report_with_query(text, model,query,prompt)
    response_text_3 = SINGLE_0_72topp_generate_report_with_query(text, model,query,prompt)

    ref_row = [query , 
               model_response, 
               response_text_1, 
               zero_shot_response_1_score, 
               response_text_2, 
               zero_shot_response_2_score, 
               response_text_3,
               zero_shot_response_3_score,
               avg_score,
               ]
    
    SINGLE_0_72topp_zero_shot_single_response_df.loc[-1] = ref_row
    SINGLE_0_72topp_zero_shot_single_response_df.index = SINGLE_0_72topp_zero_shot_single_response_df.index + 1  #shift index
    SINGLE_0_72topp_zero_shot_single_response_df = SINGLE_0_72topp_zero_shot_single_response_df.sort_index()  #sort by index

In [38]:
SINGLE_0_72topp_zero_shot_single_response_df.to_csv('FINETUNE_SINGLE_0_72topp_zero_shot_single_response_df_2.csv')

#### Getting BERTScores

In [71]:
## BERT Scores
AllBERT_Scores = pd.DataFrame()
AllBERT_Scores = pd.DataFrame(columns=['Prompt Engineering Technique','Precision Mean','Precision Stddev','Recall Mean','Recall Stddev','F1 Score Mean','F1 Score Stddev','hashcode'])

# llm_model = ['phi3:latest','llama3.1:latest']
for response_df_index in range(len(response_df_list)): 
    Response_cols = list(filter(lambda x: (('score' not in x) and ('shot_response' in x)) == True, list(response_df_list[response_df_index].columns)))
    print('Response_cols : ',Response_cols)
    ModelSummary_col = 'model_response'
    technique_name = response_df_list_names[response_df_index]
    print('technique_name : ',technique_name)
    
    
    response_df_counter=1
    for Response_col in Response_cols:
        
        if response_df_counter ==1:
            test_bert_df = response_df_list[response_df_index][[Response_col,ModelSummary_col]].copy(deep=True)
            test_bert_df[Response_col] = test_bert_df[Response_col].replace('', np.nan)
            test_bert_df = test_bert_df.dropna(subset=[Response_col])
            test_bert_df = test_bert_df.reset_index(drop=True)
            
            BERT_Scores_1 = bertscore.compute(predictions=test_bert_df[Response_col], references=test_bert_df[ModelSummary_col], lang="en")
            precision_mean_1 = statistics.mean(BERT_Scores_1['precision'])
            precision_stdev_1 = statistics.stdev(BERT_Scores_1['precision'])
            recall_mean_1 = statistics.mean(BERT_Scores_1['recall'])
            recall_stdev_1 = statistics.stdev(BERT_Scores_1['recall'])
            f1_mean_1 = statistics.mean(BERT_Scores_1['f1'])
            f1_stdev_1 = statistics.stdev(BERT_Scores_1['f1'])
            hash_code_1 = BERT_Scores_1['hashcode']
        else:
            test_bert_df = response_df_list[response_df_index][[Response_col,ModelSummary_col]].copy(deep=True)
            test_bert_df[Response_col] = test_bert_df[Response_col].replace('', np.nan)
            test_bert_df = test_bert_df.dropna(subset=[Response_col])
            test_bert_df = test_bert_df.reset_index(drop=True)
            
            BERT_Scores_2 = bertscore.compute(predictions=test_bert_df[Response_col], references=test_bert_df[ModelSummary_col], lang="en")
            precision_mean_2 = statistics.mean(BERT_Scores_2['precision'])
            precision_stdev_2 = statistics.stdev(BERT_Scores_2['precision'])
            recall_mean_2 = statistics.mean(BERT_Scores_2['recall'])
            recall_stdev_2 = statistics.stdev(BERT_Scores_2['recall'])
            f1_mean_2 = statistics.mean(BERT_Scores_2['f1'])
            f1_stdev_2 = statistics.stdev(BERT_Scores_2['f1'])
            hash_code_2 = BERT_Scores_2['hashcode']
        response_df_counter+=1
        
    
    print('BERT_Scores_1 : ',BERT_Scores_1)
    print('BERT_Scores_2 : ',BERT_Scores_2)
    
    ref_row = [technique_name , 
               (precision_mean_1 + precision_mean_2)/2,
               (precision_stdev_1 + precision_stdev_2)/2,
               (recall_mean_1 + recall_mean_2)/2,
               (recall_stdev_1 + recall_stdev_2)/2,
               (f1_mean_1 + f1_mean_2)/2,
               (f1_stdev_1 + f1_stdev_2)/2,
               hash_code_1
               ]
    
    print('ref_row : ',ref_row)
    
    AllBERT_Scores.loc[-1] = ref_row
    AllBERT_Scores.index = AllBERT_Scores.index + 1  #shift index
    AllBERT_Scores = AllBERT_Scores.sort_index()  #sort by index
    

Response_cols :  ['zero_shot_response_1', 'zero_shot_response_2']
technique_name :  zero_STEP_WITH_EX_COT_SYS_single_response_df
BERT_Scores_1 :  {'precision': [0.8748749494552612, 0.8916327357292175, 0.9043223857879639, 0.8711660504341125, 0.8965637683868408, 0.8562110662460327, 0.8699185252189636, 0.8661855459213257, 0.9063277840614319, 0.8756944537162781], 'recall': [0.8488946557044983, 0.8533327579498291, 0.8681201934814453, 0.8589116334915161, 0.8532718420028687, 0.8507874011993408, 0.8593361377716064, 0.8591965436935425, 0.8580129742622375, 0.8469337224960327], 'f1': [0.8616889715194702, 0.8720623850822449, 0.8858515620231628, 0.8649954199790955, 0.8743822574615479, 0.8534905910491943, 0.8645949363708496, 0.8626769185066223, 0.8815088272094727, 0.8610740303993225], 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.48.2)'}
BERT_Scores_2 :  {'precision': [0.875113308429718, 0.9153162837028503, 0.8849759101867676, 0.8580702543258667, 0.874768078327179, 0.8383445143699

In [49]:
AllBERT_Scores

,Prompt Engineering Technique,Precision Mean,Precision Stddev,Recall Mean,Recall Stddev,F1 Score Mean,F1 Score Stddev,hashcode
0,SINGLE_0_96T_zero_shot_single_response_df,0.885445,0.024749,0.862300,0.021424,0.873681,0.022351,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
1,SINGLE_0_64T_zero_shot_single_response_df,0.886623,0.017167,0.866169,0.013650,0.876246,0.014526,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
2,SINGLE_48topk_zero_shot_single_response_df,0.887272,0.020541,0.865990,0.010636,0.876447,0.014518,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
3,SINGLE_32topk_zero_shot_single_response_df,0.887839,0.019354,0.864531,0.011968,0.875982,0.014471,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
4,SINGLE_1_08topp_zero_shot_single_response_df,0.888724,0.021692,0.866987,0.016908,0.877669,0.018271,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
5,SINGLE_0_72topp_zero_shot_single_response_df,0.891511,0.020963,0.867143,0.014506,0.879098,0.016252,roberta-large_L17_no-idf_version=0.3.12(hug_tr...
